### Refining quadrilateral and hexahedral element meshes
https://gitlab.onelab.info/gmsh/gmsh/-/issues/1381

In [ ]:
import gmsh
import numpy as np

cl = 9
cg = (0, 0)
DIM = 2

In [ ]:
def get_square_points(square):
    _, adjancecies_down = gmsh.model.getAdjacencies(DIM, square)
    _, pt_tags_adj_down = gmsh.model.getAdjacencies(DIM - 1, adjancecies_down[2])
    return pt_tags_adj_down

def add_line_between_squares(square_0_pts, square_1_pts):
    line_0_0 = gmsh.model.occ.addLine(startTag=square_0_pts[0], endTag=square_1_pts[0])
    line_0_1 = gmsh.model.occ.addLine(startTag=square_0_pts[1], endTag=square_1_pts[1])
    return line_0_0, line_0_1

def add_refinement_connectivity(square_0, square_1, squares_n2, squares_n22):
    square_0_pts = get_square_points(square_0)
    square_1_pts = get_square_points(square_1)
    
    line_0_1, line_0_2 = add_line_between_squares(square_0_pts, square_1_pts)

    squares_n2_pts = np.empty((len(squares_n2), 2), dtype=int)
    for i, square in enumerate(squares_n2):
        pt_tags_adj_down = get_square_points(square)
        squares_n2_pts[i] = pt_tags_adj_down

    squares_n22_pts = np.empty((len(squares_n22), 2), dtype=int)
    for i, square in enumerate(squares_n22):
        pt_tags_adj_down = get_square_points(square)
        squares_n22_pts[i] = pt_tags_adj_down


    lines_connectors = np.empty((len(squares_n2), 2), dtype=int)
    for i in range(3):
        line_0_1, line_0_2 = add_line_between_squares(squares_n2_pts[i], squares_n22_pts[i])
        lines_connectors[i] = [line_0_1, line_0_2]
    gmsh.model.occ.synchronize()
    return lines_connectors

def create_square_geom(cg, cl):
    square_0 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl, cl)
    square_1 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/6, 0, cl/3, cl/3)


    square_11 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl/3, cl/3)
    square_12 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/2, 0, cl/3, cl/3)
    square_13 = gmsh.model.occ.addRectangle(cg[0] + cl/6, cg[1] - cl/2, 0, cl/3, cl/3)
    squares_n2 = [square_11, square_12, square_13]

    square_111 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl/9, cl/9)
    square_112 = gmsh.model.occ.addRectangle(cg[0] - 3.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_113 = gmsh.model.occ.addRectangle(cg[0] - 2.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_211 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/2, 0, cl/9, cl/9)
    square_212 = gmsh.model.occ.addRectangle(cg[0] - 0.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_213 = gmsh.model.occ.addRectangle(cg[0] + 0.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_311 = gmsh.model.occ.addRectangle(cg[0] + cl/6, cg[1] - cl/2, 0, cl/9, cl/9)
    square_312 = gmsh.model.occ.addRectangle(cg[0] + 2.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_313 = gmsh.model.occ.addRectangle(cg[0] + 3.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_122 = gmsh.model.occ.addRectangle(cg[0] - 3.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    square_222 = gmsh.model.occ.addRectangle(cg[0] - 0.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    square_322 = gmsh.model.occ.addRectangle(cg[0] + 2.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    squares_n22 = [square_122, square_222, square_322]
    return square_0, square_1, squares_n2, squares_n22

def remove_square_surfaces():
    for plane in gmsh.model.getEntities(2):
        gmsh.model.occ.remove([plane])
    gmsh.model.occ.synchronize()

In [ ]:
# Create a mesh of a square
gmsh.initialize()

# create base geometry
square_0, square_1, squares_n2, squares_n22 = create_square_geom(cg, cl)
gmsh.model.occ.synchronize()
# add refinement connectivity to decrease the mesh size
lines_connectors = add_refinement_connectivity(square_0, square_1, squares_n2, squares_n22)
remove_square_surfaces()

for line in gmsh.model.getEntities(1):
    gmsh.model.mesh.setTransfiniteCurve(line[1], 1)

# for surf in gmsh.model.getEntities(2):
#     gmsh.model.mesh.setTransfiniteSurface(surf[1])
# gmsh.model.mesh.setRecombine(2, 1)
# gmsh.model.mesh.generate(2)
gmsh.fltk.run()
gmsh.finalize()